In [8]:
import requests
import cv2 #OpenCV(Open Source Computer Vision Library)의 파이썬 바인딩
import pytesseract
from PIL import Image
import numpy as np
import time
import logging
import json

logging.basicConfig(level=logging.INFO)

def load_config_json(file_path): #Json파일을 로드
    try:
        with open(file_path, 'r') as file:
            config = json.load(file)
        return config
    except FileNotFoundError:
        logging.error(f"파일을 찾을 수 없습니다: {file_path}")
    except json.JSONDecodeError as e:
        logging.error(f"JSON 구문 오류 발생: {e}")
    except Exception as e:
        logging.error(f"오류 발생: {e}")

def fetch_data_with_retries(url, headers, max_retries=5, backoff_factor=1): #API 호출을 재시도하며 데이터를 가져오는 코드
    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # HTTP 오류가 발생하면 예외를 던집니다.
            return response.json()  # JSON 응답을 반환합니다.
        except requests.exceptions.HTTPError as err:
            if response.status_code == 429:  # Too Many Requests
                logging.warning(f"요청 제한 초과. {backoff_factor ** retries}초 후 재시도...")
                time.sleep(backoff_factor ** retries)
                retries += 1
            else:
                raise err
    raise Exception("최대 재시도 횟수 초과")

def get_user_num(nick, base_url, headers): #nick으로 userNum추출
    info_url = f"{base_url}user/nickname?query={nick}"
    try:
        info_data = fetch_data_with_retries(info_url, headers) 
        userNum = info_data.get("user", {}).get("userNum")
        return userNum
    except Exception as e:
        logging.error(f"사용자 번호 추출 오류: {e}")

def get_character_info(characterCode, base_url_v2, headers): #캐릭터 정보 추출(DB?)
    try:
        character_url = f"{base_url_v2}data/Character"
        character_data = fetch_data_with_retries(character_url, headers).get("data", [])
        character_info = character_data[characterCode - 1] if characterCode - 1 < len(character_data) else None
        return character_info
    except Exception as e:
        logging.error(f"캐릭터 정보 추출 오류: {e}")   

def get_user_stats(userNum, base_url, headers, seasonId): #userNum으로 seosonID에 해당하는 사용캐릭터 정보추출
    
    stats_url = f"{base_url}user/stats/{userNum}/{seasonId}"
    try:
        stats_data = fetch_data_with_retries(stats_url, headers).get('userStats', [])
        if not stats_data:
            return []
        return [stat for stat in stats_data[0].get('characterStats', []) if 'characterCode' in stat]
        #mostCharacter_list = []
        #character_stats = stats_data[0].get('characterStats', [])
        #for stat in character_stats:
            #if 'characterCode' in stat:
                #mostCharacter_list.append(stat)

        #return(mostCharacter_list)
    except Exception as e:
        logging.error(f"사용자 통계 추출 오류: {e}")

def match_chatacter(nick, base_url, base_url_v2, headers, seasonId): #nick으로 추출한 사용캐릭터 정보와 캐릭터 정보를 매치하여 반환
    userNum = get_user_num(nick, base_url, headers)

    if not userNum: #
        logging.error(f"사용자 {nick}을(를) 찾을 수 없습니다.")
        return None, None #
    
    mostCharacter_list = get_user_stats(userNum, base_url, headers, seasonId)

    if not mostCharacter_list: #
        logging.error(f"사용자 {nick}의 캐릭터 통계를 찾을 수 없습니다.")
        return None, None
    
    character_info_list = [] #
    for mostCharacter in mostCharacter_list:
        characterCode = mostCharacter['characterCode']
        character_info = get_character_info(characterCode, base_url_v2, headers)
        if character_info:
            character_info_list.append(character_info)
        else:
            logging.warning(f"캐릭터 코드 {characterCode}에 대한 정보가 없습니다.")
    return character_info_list, mostCharacter_list

def get_user_names(image_path): #Tesseract를 통해 캡쳐된 사진에서 NickList 추출
    nick_list = []
    pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

    try:
        pil_image = Image.open(image_path)
        image = np.array(pil_image) #PIL 이미지를 NumPy 배열로 변환하면 OpenCV와 같은 이미지 처리 라이브러리와의 호환성, 고속 연산, 배열 조작의 유연성 등의 장점이 있지만, 메모리 사용량 증가와 변환 오버헤드, 형식 차이 등 단점이 있음
        #pil_image = Image.fromarray(image)

        start_len: int = 1080
        start_hgt: int = 930
        nameSpace_interval: int = 284
        nameSpace_len: int = 135
        nameSpace_hgt: int = 20
        #(1080,930,1215,950) 초기값

        for i in range(3):
            crop_area = (start_len+(nameSpace_interval*i),start_hgt,start_len+(nameSpace_interval*i)+nameSpace_len,start_hgt    +nameSpace_hgt) # 특정 영역 자르기 (left, upper, right, lower)
            cropped_image = image[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2]]
            result = pytesseract.image_to_string(cropped_image, lang='kor+eng+jpn')
            if result:
                nick_list.append(result.replace('\n',''))
            else:
                #cv2.imshow(f'Cropped Image {i+1}', cropped_image)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                logging.warning(f"크롭된 이미지 {i+1}에서 텍스트를 추출할 수 없습니다.")
                nick_list.append(None)            
    except Exception as e:
        logging.error(f"이미지 처리 오류: {e}")
    return nick_list

def main():
    config_file = 'C:/Users/Moon/Desktop/ER/ER-main/01. Search/config.json'
    config = load_config_json(config_file)
    if not config:
        logging.error("설정을 로드할 수 없습니다.")
        return
    
    api_key = config.get("api_key")
    base_url = config.get("base_url")
    base_url_v2 = config.get("base_url_v2")

    if not api_key:
        print("API_KEY가 설정되지 않았습니다.")
        return
    if not base_url:
        print("base_url이 설정되지 않았습니다.")
        return
    if not base_url_v2:
        print("base_url_v2가 설정되지 않았습니다.")
        return
    headers = {"x-api-key": api_key}
    seasonId = 25 #EA9*2 = 18  정규3*2 = 6
    #matchingTeamMode = 3 #스쿼드 3
    image_path = 'C:/Start/python_basic/05.Tesseract/Image/test1.jpg'
    nick_list = get_user_names(image_path)
    for nick in nick_list:
        character_info_list, mostCharacter_list = match_chatacter(nick, base_url, base_url_v2, headers, seasonId)
        #print(mostCharacter_list)
        if character_info_list:
            logging.info(f'닉네임: {nick}')
            for idx, character_info in enumerate(character_info_list[:3], 1): #첫 3 개의 요소를 순회하면서, 인덱스를 1부터 시작
                most_character = mostCharacter_list[idx-1] if idx-1 < len(mostCharacter_list) else {}
                logging.info(f'{idx}_캐릭터: {character_info.get("name", "알 수 없음")} '
                    f'({most_character.get("usages", 0)} 게임, {most_character.get("wins", 0)} 승, '
                    f'Top3: {most_character.get("top3", 0)}, 평균 등수: {most_character.get("averageRank", 0)})')

if __name__ == "__main__":
    main()

ERROR:root:파일을 찾을 수 없습니다: C:/Users/Moon/Desktop/ER/ER-main/01. Search/config.json
ERROR:root:설정을 로드할 수 없습니다.


기본

In [3]:
import requests
import cv2
import pytesseract
from PIL import Image
import numpy as np
import time
import logging
import os
import json
import sys
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow
from PyQt5.QtGui import QFont, QKeyEvent
import glob
from datetime import datetime

logging.basicConfig(level=logging.INFO)

class ConfigLoader:
    @staticmethod
    def load_config(file_path):
        try:
            with open(file_path, 'r') as file:
                return json.load(file)
        except FileNotFoundError:
            logging.error(f"파일을 찾을 수 없습니다: {file_path}")
        except json.JSONDecodeError as e:
            logging.error(f"JSON 구문 오류 발생: {e}")
        except Exception as e:
            logging.error(f"오류 발생: {e}")
        return None

class APIClient:
    def __init__(self, base_url, headers, max_retries=5, backoff_factor=1):
        self.base_url = base_url
        self.headers = headers
        self.max_retries = max_retries
        self.backoff_factor = backoff_factor

    def fetch_data(self, endpoint):
        retries = 0
        while retries < self.max_retries:
            try:
                response = requests.get(f"{self.base_url}{endpoint}", headers=self.headers)
                response.raise_for_status()
                return response.json()
            except requests.exceptions.HTTPError as err:
                if response.status_code == 429:
                    logging.warning(f"요청 제한 초과. {self.backoff_factor ** retries}초 후 재시도...")
                    time.sleep(self.backoff_factor ** retries)
                    retries += 1
                else:
                    logging.error(f"HTTP 에러 발생: {err}")
                    raise err
            except Exception as e:
                logging.error(f"데이터 요청 오류: {e}")
        raise Exception("최대 재시도 횟수 초과")

    def get_user_num(self, nick):
        try:
            data = self.fetch_data(f"user/nickname?query={nick}")
            return data.get("user", {}).get("userNum")
        except Exception as e:
            logging.error(f"사용자 번호 추출 오류: {e}")
            return None

    def get_character_info(self, character_code):
        try:
            data = self.fetch_data("data/Character")
            return data.get("data", [])[character_code - 1] if character_code - 1 < len(data.get("data", [])) else None
        except Exception as e:
            logging.error(f"캐릭터 정보 추출 오류: {e}")
            return None

    def get_user_stats(self, user_num, season_id):
        try:
            data = self.fetch_data(f"user/stats/{user_num}/{season_id}")
            return [stat for stat in data.get('userStats', [])[0].get('characterStats', []) if 'characterCode' in stat]
        except Exception as e:
            logging.error(f"사용자 통계 추출 오류: {e}")
            return []

class ImageProcessor:
    @staticmethod
    def get_latest_file(directory_path, extension='*.png'):
        files = glob.glob(os.path.join(directory_path, extension))
        latest_file = max(files, key=os.path.getctime, default=None)
        return latest_file

    @staticmethod
    def extract_usernames(image_path,start_len,start_hgt,nameSpace_interval,nameSpace_len,nameSpace_hgt):
        nick_list = []
        pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

        try:
            pil_image = Image.open(image_path)
            image = np.array(pil_image)

            #start_len = 1880
            #start_hgt = 1240
            #nameSpace_interval = 380
            #nameSpace_len = 180
            #nameSpace_hgt = 30

            for i in range(3):
                if i == 0: #내 정보는 필요없으니까 처리속도 향상을 위해 제거
                    continue
                crop_area = (
                    start_len + (nameSpace_interval * i),
                    start_hgt,
                    start_len + (nameSpace_interval * i) + nameSpace_len,
                    start_hgt + nameSpace_hgt
                )
                cropped_image = image[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2]]
                result = pytesseract.image_to_string(cropped_image, lang='kor+eng+jpn').strip()
                if result:
                    nick_list.append(result)
                else:
                    #cv2.imshow(f'Cropped Image {i+1}', cropped_image)
                    #cv2.waitKey(0)
                    #cv2.destroyAllWindows()
                    logging.warning(f"크롭된 이미지 {i+1}에서 텍스트를 추출할 수 없습니다.")
                    nick_list.append(None)
        except Exception as e:
            logging.error(f"이미지 처리 오류: {e}")
        return nick_list

class OverlayWindow(QMainWindow):
    def __init__(self, overlay_text, color = "Black", font_size = 20, font = "Arial"):
        super().__init__()
        self.setWindowFlags(Qt.FramelessWindowHint | Qt.WindowStaysOnTopHint | Qt.WindowTransparentForInput)
        self.setAttribute(Qt.WA_TranslucentBackground)

        self.label = QLabel(self)
        self.label.setText(overlay_text)
        self.label.setFont(QFont(font, font_size))
        self.label.setStyleSheet(f"color: {color}")
        self.label.adjustSize()

        self.setGeometry(2000, 100, self.label.width(), self.label.height())

    def keyPressEvent(self, event: QKeyEvent):
        if event.key() == Qt.Key_Escape:
            self.close()

class CharacterMatcher:
    def __init__(self, api_client, api_client_v2, season_id):
        self.api_client = api_client
        self.api_client_v2 = api_client_v2
        self.season_id = season_id

    def match_character(self, nick):
        user_num = self.api_client.get_user_num(nick)
        if not user_num:
            logging.error(f"사용자 {nick}을(를) 찾을 수 없습니다.")
            return None, None

        most_character_list = self.api_client.get_user_stats(user_num, self.season_id)
        if not most_character_list:
            logging.error(f"사용자 {nick}의 캐릭터 통계를 찾을 수 없습니다.")
            return None, None

        character_info_list = []
        for most_character in most_character_list:
            character_info = self.api_client_v2.get_character_info(most_character['characterCode'])
            if character_info:
                character_info_list.append(character_info)
            else:
                logging.warning(f"캐릭터 코드 {most_character['characterCode']}에 대한 정보가 없습니다.")
        return character_info_list, most_character_list

def main():
    main_directory = 'C:/Start/ER/01. Search/'
    config_file = main_directory+'config.json'
    config = ConfigLoader.load_config(config_file)

    if not config:
        logging.error("설정 파일을 로드할 수 없습니다.")
        return

    api_key = config.get("api_key")
    base_url = config.get("base_url")
    base_url_v2 = config.get("base_url_v2")
    season_id = 25

    if not api_key or not base_url or not base_url_v2:
        logging.error("필수 설정 값이 없습니다.")
        return

    headers = {"x-api-key": api_key}
    api_client = APIClient(base_url, headers)
    api_client_v2 = APIClient(base_url_v2, headers)

    character_matcher = CharacterMatcher(api_client, api_client_v2, season_id)

    image_directory_path = main_directory+'Image'
    image_path = ImageProcessor.get_latest_file(image_directory_path)
    start_len, start_hgt, nameSpace_interval, nameSpace_len, nameSpace_hgt = 1880, 1240, 380, 180, 30

    if not image_path:
        logging.error("이미지 파일을 찾을 수 없습니다.")
        return

    nick_list = ImageProcessor.extract_usernames(image_path,start_len,start_hgt,nameSpace_interval,nameSpace_len,nameSpace_hgt)
    
    overlay_text = ""
    color, font_size, font = "GREEN", 20, "Arial"

    for nick in nick_list:
        character_info_list, most_character_list = character_matcher.match_character(nick)
        if character_info_list:
            overlay_text += f'닉네임: {nick}\n'
            for idx, character_info in enumerate(character_info_list[:3], 1):
                most_character = most_character_list[idx - 1] if idx - 1 < len(most_character_list) else {}
                overlay_text += (f'{idx}_캐릭터: {character_info.get("name", "알 수 없음")} '
                                 f'({most_character.get("usages", 0)} 게임, {most_character.get("wins", 0)} 승, '
                                 f'Top3: {most_character.get("top3", 0)}, 평균 등수: {most_character.get("averageRank", 0)})\n')
            overlay_text += "\n"

    if overlay_text:
        app = QApplication(sys.argv)
        overlay = OverlayWindow(overlay_text, color, font_size, font)
        overlay.show()
        sys.exit(app.exec_())

if __name__ == "__main__":
    main()

SystemExit: 0

클래스화

In [6]:
import requests
import pytesseract
from PIL import Image
import numpy as np
import time
import logging
import os
import json
import sys
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow
from PyQt5.QtGui import QFont, QKeyEvent
import glob
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO)

class ConfigLoader:
    @staticmethod
    def load_config(file_path):
        try:
            with open(file_path, 'r') as file:
                return json.load(file)
        except FileNotFoundError:
            logging.error(f"파일을 찾을 수 없습니다: {file_path}")
        except json.JSONDecodeError as e:
            logging.error(f"JSON 구문 오류 발생: {e}")
        except Exception as e:
            logging.error(f"오류 발생: {e}")
        return None

class APIClient:
    def __init__(self, base_url, headers, max_retries=5, backoff_factor=1):
        self.base_url = base_url
        self.headers = headers
        self.max_retries = max_retries
        self.backoff_factor = backoff_factor
        self.cache = {}

    def fetch_data(self, endpoint):
        if endpoint in self.cache:
            return self.cache[endpoint]

        retries = 0
        while retries < self.max_retries:
            try:
                response = requests.get(f"{self.base_url}{endpoint}", headers=self.headers)
                response.raise_for_status()
                data = response.json()
                self.cache[endpoint] = data
                return data
            except requests.exceptions.HTTPError as err:
                if response.status_code == 429:
                    logging.warning(f"요청 제한 초과. {self.backoff_factor ** retries}초 후 재시도...")
                    time.sleep(self.backoff_factor ** retries)
                    retries += 1
                else:
                    logging.error(f"HTTP 에러 발생: {err}")
                    raise err
            except Exception as e:
                logging.error(f"데이터 요청 오류: {e}")
        raise Exception("최대 재시도 횟수 초과")

    def get_user_num(self, nick):
        try:
            data = self.fetch_data(f"user/nickname?query={nick}")
            return data.get("user", {}).get("userNum")
        except Exception as e:
            logging.error(f"사용자 번호 추출 오류: {e}")
            return None

    def get_character_info(self, character_code):
        try:
            data = self.fetch_data("data/Character")
            return data.get("data", [])[character_code - 1] if character_code - 1 < len(data.get("data", [])) else None
        except Exception as e:
            logging.error(f"캐릭터 정보 추출 오류: {e}")
            return None

    def get_user_stats(self, user_num, season_id):
        try:
            data = self.fetch_data(f"user/stats/{user_num}/{season_id}")
            return [stat for stat in data.get('userStats', [])[0].get('characterStats', []) if 'characterCode' in stat]
        except Exception as e:
            logging.error(f"사용자 통계 추출 오류: {e}")
            return []

class ImageProcessor:
    @staticmethod
    def get_latest_file(directory_path, extension='*.png'):
        files = glob.glob(os.path.join(directory_path, extension))
        latest_file = max(files, key=os.path.getctime, default=None)
        return latest_file

    @staticmethod
    def extract_usernames(image_path, start_len, start_hgt, nameSpace_interval, nameSpace_len, nameSpace_hgt):
        nick_list = []
        pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

        try:
            pil_image = Image.open(image_path)
            image = np.array(pil_image)

            for i in range(3):
                if i == 0: #내 정보는 필요없으니까 처리속도 향상을 위해 제거
                    continue
                crop_area = (
                    start_len + (nameSpace_interval * i),
                    start_hgt,
                    start_len + (nameSpace_interval * i) + nameSpace_len,
                    start_hgt + nameSpace_hgt
                )
                cropped_image = image[crop_area[1]:crop_area[3], crop_area[0]:crop_area[2]]
                result = pytesseract.image_to_string(cropped_image, lang='kor+eng+jpn').strip()
                if result:
                    nick_list.append(result)
                else:
                    logging.warning(f"크롭된 이미지 {i+1}에서 텍스트를 추출할 수 없습니다.")
                    nick_list.append(None)
        except Exception as e:
            logging.error(f"이미지 처리 오류: {e}")
        finally:
            pil_image.close()  # Ensure the image file is closed after processing
        return nick_list

class OverlayWindow(QMainWindow):
    def __init__(self, overlay_text, color="Black", font_size=20, font="Arial"):
        super().__init__()
        self.setWindowFlags(Qt.FramelessWindowHint | Qt.WindowStaysOnTopHint | Qt.WindowTransparentForInput)
        self.setAttribute(Qt.WA_TranslucentBackground)

        self.label = QLabel(self)
        self.label.setText(overlay_text)
        self.label.setFont(QFont(font, font_size))
        self.label.setStyleSheet(f"color: {color}")
        self.label.adjustSize()

        self.setGeometry(2000, 100, self.label.width(), self.label.height())

    def keyPressEvent(self, event: QKeyEvent):
        if event.key() == Qt.Key_Escape:
            self.close()

class CharacterMatcher:
    def __init__(self, api_client, api_client_v2, season_id):
        self.api_client = api_client
        self.api_client_v2 = api_client_v2
        self.season_id = season_id

    def match_character(self, nick):
        user_num = self.api_client.get_user_num(nick)
        if not user_num:
            logging.error(f"사용자 {nick}을(를) 찾을 수 없습니다.")
            return None, None

        most_character_list = self.api_client.get_user_stats(user_num, self.season_id)
        if not most_character_list:
            logging.error(f"사용자 {nick}의 캐릭터 통계를 찾을 수 없습니다.")
            return None, None

        character_info_list = []
        with ThreadPoolExecutor(max_workers=3) as executor:  # Adjust the number of workers as needed
            futures = [executor.submit(self.api_client_v2.get_character_info, most_character['characterCode'])
                       for most_character in most_character_list]
            for future in as_completed(futures):
                try:
                    info = future.result()
                    if info:
                        character_info_list.append(info)
                except Exception as e:
                    logging.error(f"캐릭터 정보 가져오기 오류: {e}")

        return character_info_list, most_character_list

def main():
    main_directory = 'C:/Start/ER/01. Search/'
    config_file = os.path.join(main_directory, 'config.json')
    config = ConfigLoader.load_config(config_file)

    if not config:
        logging.error("설정 파일을 로드할 수 없습니다.")
        return

    api_key = config.get("api_key")
    base_url = config.get("base_url")
    base_url_v2 = config.get("base_url_v2")
    season_id = 25

    if not api_key or not base_url or not base_url_v2:
        logging.error("필수 설정 값이 없습니다.")
        return

    headers = {"x-api-key": api_key}
    api_client = APIClient(base_url, headers)
    api_client_v2 = APIClient(base_url_v2, headers)

    character_matcher = CharacterMatcher(api_client, api_client_v2, season_id)

    image_directory_path = os.path.join(main_directory, 'Image')
    image_path = ImageProcessor.get_latest_file(image_directory_path)
    start_len, start_hgt, nameSpace_interval, nameSpace_len, nameSpace_hgt = 1880, 1240, 380, 180, 30

    if not image_path:
        logging.error("이미지 파일을 찾을 수 없습니다.")
        return

    nick_list = ImageProcessor.extract_usernames(image_path, start_len, start_hgt, nameSpace_interval, nameSpace_len, nameSpace_hgt)

    overlay_text = ""
    color, font_size, font = "White", 20, "Arial"

    with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust the number of workers as needed
        future_to_nick = {executor.submit(character_matcher.match_character, nick): nick for nick in nick_list}
        for future in as_completed(future_to_nick):
            nick = future_to_nick[future]
            try:
                character_info_list, most_character_list = future.result()
                if character_info_list:
                    overlay_text += f'닉네임: {nick}\n'
                    for idx, character_info in enumerate(character_info_list[:3], 1):
                        most_character = most_character_list[idx - 1] if idx - 1 < len(most_character_list) else {}
                        overlay_text += (f'{idx}_캐릭터: {character_info.get("name", "알 수 없음")} '
                                         f'({most_character.get("usages", 0)} 게임, {most_character.get("wins", 0)} 승, '
                                         f'Top3: {most_character.get("top3", 0)})\n')
                    overlay_text += "\n"
            except Exception as e:
                logging.error(f"캐릭터 매칭 오류: {e}")

    if overlay_text:
        app = QApplication(sys.argv)
        overlay = OverlayWindow(overlay_text, color, font_size, font)
        overlay.show()
        sys.exit(app.exec_())

if __name__ == "__main__":
    main()

SystemExit: 0

병렬처리